In [18]:
!pip install huggingface_hub==0.25.0
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U trl

  Using cached huggingface_hub-0.25.0-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.25.0-py3-none-any.whl (436 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.2
    Uninstalling huggingface-hub-0.31.2:
      Successfully uninstalled huggingface-hub-0.31.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.51.3 requires huggingface-hub<1.0,>=0.30.0, but you have huggingface-hub 0.25.0 which is incompatible.
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.31.2-py3-none-any.whl (484 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.0
    Uninstalling huggingface-hub-0.25.0:
      Successfully uninstalled huggingface-hub-0.25.0


In [19]:
import os
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          logging)
from peft import LoraConfig, get_peft_model
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from trl import SFTTrainer, setup_chat_format
import bitsandbytes as bnb

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("belovsasha")
login(token=hf_token)

base_model = "google/gemma-2-2b-it"
new_model = "Gemma-2-2b-it-ChatDoctor"
dataset_name = "lavita/ChatDoctor-HealthCareMagic-100k"

if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True
)

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()

    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split(".")
            if len(names) == 1:
                lora_module_names.add(names[0])
            else:
                lora_module_names.add(names[-1])

    lora_module_names.discard('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

tokenizer.chat_template = None
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [22]:
import re
from datasets import load_dataset

dataset = load_dataset(
    dataset_name,
    split="all",
    cache_dir="./cache"
)

dataset = dataset.shuffle(seed=42).select(range(2000))

def clean_text(text):
    text = re.sub(r'\b(?:www\.[^\s]+|http\S+)','',text)
    text = re.sub(r'\b(?:aCht Doctor(?:.com)?(?:.in)?|www\.(?:google|yahoo)\S*)','',text)
    text = re.sub(r'\s+', ' ',text)
    return text.strip()


def format_chat_template(row):
    cleaned_instruction = clean_text(row['instruction'])
    cleaned_input = clean_text(row['input'])
    cleaned_output = clean_text(row["output"])

    row_json = [
        {"role": "system", "content":cleaned_instruction},
        {"role": "user", "content":cleaned_input},
        {"role": "assistant", "content":cleaned_output}
    ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(format_chat_template, num_proc=4)
dataset = dataset.train_test_split(test_size=0.1)
data_collator = lambda batch: tokenizer(
    batch["text"],
    return_tensors="pt",
    padding=True,
    truncation=True
)

In [23]:
training_args = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=500,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy='steps',
    learning_rate=0.0002,
    fp16=True,
    bf16=False,
    group_by_length=True,
    load_best_model_at_end=False,
    report_to=[]
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_args,
)

model.config.use_cache = False

Converting train dataset to ChatML:   0%|          | 0/1800 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/200 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
trainer.train()

Step,Training Loss,Validation Loss
200,1.733100,2.090374
400,1.689900,2.046673
600,1.942300,2.015561
800,1.764000,1.995803


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=900, training_loss=2.0753260933028326, metrics={'train_runtime': 1553.689, 'train_samples_per_second': 1.159, 'train_steps_per_second': 0.579, 'total_flos': 6335379518648832.0, 'train_loss': 2.0753260933028326})

In [26]:
merged_model = model.merge_and_unload()
merged_model.save_pretrained(new_model)
merged_model.push_to_hub(new_model, use_temp_dir=False)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sashabeloov/Gemma-2-2b-it-ChatDoctor/commit/387132345ac28201894d7b5e34a8b93a1b5c5357', commit_message='Upload Gemma2ForCausalLM', commit_description='', oid='387132345ac28201894d7b5e34a8b93a1b5c5357', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sashabeloov/Gemma-2-2b-it-ChatDoctor', endpoint='https://huggingface.co', repo_type='model', repo_id='sashabeloov/Gemma-2-2b-it-ChatDoctor'), pr_revision=None, pr_num=None)

In [32]:
from transformers import GenerationConfig

message = [
    {"role": "system", "content": "You are a medical expert specializing in respiratory diseases.You should prescribe some medical drugs"},
    {"role": "user", "content": "I have a persistent cough, night sweats, and recent weight loss.I've been to multiple doctors with no diagnosis yet.Could these symptoms be related to tuberculosis or another serious illness? Please provide a detailed answer considering possible causes and recommended next steps.Write down medicines that can cure my illness"}
]

prompt = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(
    **inputs,
    max_length=350,
    top_k=50,
    top_p=0.85,
    temperature=0.3,
    no_repeat_ngram_size=3
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()
print(response)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


I understand your concern. It's frustrating to have symptoms that don't seem to be resolved.  Let's discuss your symptoms and possible causes.

**Possible Causes**

* **Tuberculosis (TB):**  TB is a bacterial infection that can cause a persistent dry cough, weight loss, night sweating, and fever. It can also cause chest pain, fatigue, and weakness.
* **Other Lung Infections:**  Pneumonia, bronchitis, and other lung infections can cause similar symptoms. These infections are often caused by viruses or bacteria.
    * **Viral Pneumonia:**  Common cold, influenza, and respiratory syncytial virus (RSV) can cause pneumonia.
        * Symptoms: Cough, fever, chills, shortness of breath, chest pain.
   * **Bacterial Pneumonia:** More serious than viral pneumonia, bacterial pneumonia can cause chest discomfort, fever and chills, cough, and difficulty breathing.
       * Symptoms include fever, cough with phlegm, chest discomfort.
  * **Asthma:**  Asthma can cause coughing, wheezing, and shortn